# Hunter2Libib
This is a notebook to convert data from Book Hunter toLibIb. 
Here we translate a book collection from Book Hunter to a CSV to import in LibIb. 

In [4]:
import pandas as pd
import isbnlib as isbn
import urllib3
import re


FILE = "./books.csv"
FILE_CACHE = "./books_libib_cache.csv"
FILE_IMPORT = "./books_libib_import.csv"

COLUMS_EXPORT = ["Description", 
                 "Group", 
                 "Authors", 
                 "Status",
                 "Rating",
                 "Title",
                 "isbn_13"]

In [5]:
b_h_dataframe = pd.read_csv(FILE)

# Analize Book Hunter format

In [92]:
b_h_dataframe.head()

,Z_PK,Z_ENT,Z_OPT,ZSTATUS,ZISINWISHLIST,ZISBORROWED,ZGROUP,ZDATEACQUIRED,ZDATEMODIFIED,ZLASTTIMEREAD,...,ZASIN,ZFORMAT,ZLENGTH,ZISBN,ZEDITOR,ZTRANSLATOR,ZGENRE,ZLANGUAGES,ZAUTHOR,ZCOLLECTION
0,2,1,1,0,0,0,NaN,307404000,307454400,NaN,...,NaN,NaN,151,NaN,Alianza Editorial,NaN,Novela,Castelan,"Goethe, Johann Wolfgang",NaN
1,3,1,1,0,0,0,NaN,307404000,307454400,NaN,...,NaN,NaN,350,NaN,Vergara,NaN,Biograficas,Castelan,"Noll, Richard",NaN
2,4,1,1,0,0,0,NaN,307404000,307454400,NaN,...,NaN,NaN,202,NaN,Anagrama,NaN,Ciencia-Ficcion,Castelan,"Auster, Paul",NaN
3,6,1,1,0,0,0,NaN,307404000,307454400,NaN,...,NaN,NaN,260,NaN,Civitas,NaN,Economia,Castelan,"Perez Touriño, Emilio",Estudios Y Monografiias
4,7,1,1,0,0,0,NaN,307404000,307454400,NaN,...,NaN,NaN,127,NaN,Tribuna,NaN,Ensaio,Castelan,Anonimo,NaN


In [93]:
b_h_dataframe.describe()

,Z_PK,Z_ENT,Z_OPT,ZSTATUS,ZISINWISHLIST,ZISBORROWED,ZGROUP,ZDATEACQUIRED,ZDATEMODIFIED,ZLASTTIMEREAD,ZPUBLISHDATE,ZPUBLISHPLACE,ZTAGS,ZPERSONALID,ZLENGTH
count,1394.000000,1394,1394.000000,1394.000000,1394,1394,0,1.339000e+03,1.394000e+03,0,2.990000e+02,0,0,0,1053.000000
mean,772.290531,1,1.373745,1.395983,0,0,NaN,3.549326e+08,3.630976e+08,NaN,-1.566588e+09,NaN,NaN,NaN,320.396961
std,445.374656,0,0.596881,1.873497,0,0,NaN,8.818051e+07,8.739779e+07,NaN,7.192256e+09,NaN,NaN,NaN,189.165061
min,2.000000,1,1.000000,0.000000,0,0,NaN,3.074040e+08,3.074544e+08,NaN,-5.998406e+10,NaN,NaN,NaN,8.000000
25%,383.250000,1,1.000000,0.000000,0,0,NaN,3.074040e+08,3.074544e+08,NaN,3.170520e+07,NaN,NaN,NaN,200.000000
50%,770.500000,1,1.000000,0.000000,0,0,NaN,3.074040e+08,3.074544e+08,NaN,2.851164e+08,NaN,NaN,NaN,288.000000
75%,1161.750000,1,2.000000,4.000000,0,0,NaN,3.635640e+08,3.951468e+08,NaN,3.630024e+08,NaN,NaN,NaN,400.000000
max,1542.000000,1,6.000000,4.000000,0,0,NaN,6.233688e+08,6.234115e+08,NaN,5.226264e+08,NaN,NaN,NaN,1998.000000


In [94]:
b_h_dataframe.columns

Index(['Z_PK', 'Z_ENT', 'Z_OPT', 'ZSTATUS', 'ZISINWISHLIST', 'ZISBORROWED',
       'ZGROUP', 'ZDATEACQUIRED', 'ZDATEMODIFIED', 'ZLASTTIMEREAD',
       'ZPUBLISHDATE', 'ZDATABASEID', 'ZLOCATION', 'ZRATING', 'ZTITLE',
       'ZVOTERATING', 'ZSUMMARY', 'ZPUBLISHPLACE', 'ZTAGS', 'ZILLUSTRATOR',
       'ZPERSONALID', 'ZASIN', 'ZFORMAT', 'ZLENGTH', 'ZISBN', 'ZEDITOR',
       'ZTRANSLATOR', 'ZGENRE', 'ZLANGUAGES', 'ZAUTHOR', 'ZCOLLECTION'],
      dtype='object')

In [95]:
b_h_dataframe[["Z_OPT",
               "ZSTATUS",
               "ZTAGS",
               "ZRATING",
               "ZISBN",
               "ZASIN",
               "ZGENRE",
               "ZTITLE",
               "ZAUTHOR"]].head()

,Z_OPT,ZSTATUS,ZTAGS,ZRATING,ZISBN,ZASIN,ZGENRE,ZTITLE,ZAUTHOR
0,1,0,NaN,NaN,NaN,NaN,Novela,Penas Del Joven Werther,"Goethe, Johann Wolfgang"
1,1,0,NaN,NaN,NaN,NaN,Biograficas,"Jung, El Cristo Ario","Noll, Richard"
2,1,0,NaN,NaN,NaN,NaN,Ciencia-Ficcion,El Pais de Las Ultimas Cosas,"Auster, Paul"
3,1,0,NaN,NaN,NaN,NaN,Economia,Infraestructuras Y Desarrollo Regional,"Perez Touriño, Emilio"
4,1,0,NaN,NaN,NaN,NaN,Ensaio,Como Adivinar El Horoscopo,Anonimo


### Z_ OPT 

Values 1-6

I have not found meaning

In [96]:
b_h_dataframe.loc[b_h_dataframe["Z_OPT"] == 2][["Z_OPT","ZTITLE"]].head()

,Z_OPT,ZTITLE
5,2,Como Morimos
14,2,Memorias de Adriano
18,2,Enemigo Del Sistema
19,2,Un Guijarro En El Cielo
26,2,La Clave


## Z_STATUS
 * 0.-Unread 
 * 1.-To Read 
 * 2.-Read 
 * 3.-Unfinish 
 * 4.- Read

In [97]:
b_h_dataframe.loc[b_h_dataframe["ZSTATUS"] ==4][["ZSTATUS","ZTITLE"]].head()

,ZSTATUS,ZTITLE
5,4,Como Morimos
7,4,Contra El Infinito
14,4,Memorias de Adriano
18,4,Enemigo Del Sistema
19,4,Un Guijarro En El Cielo


### Z_TAGS
No values

In [98]:
b_h_dataframe.loc[b_h_dataframe["ZTAGS"].notnull()][["ZTAGS","ZTITLE"]]

,ZTAGS,ZTITLE


### Z_RATING 
Values:1-6

Book Rating

In [6]:
b_h_dataframe.loc[b_h_dataframe["ZRATING"].notnull()][["ZRATING","ZTITLE"]].sort_values(
                                                            by=["ZRATING"],  
                                                            ascending=False)

,ZRATING,ZTITLE
1314,Sokyo Ono,Sintoismo. La via de los kami
1112,6,wild card II
1105,6,How to Create a Mind: The Secret of Human Thou...
1045,6,Los juegos del hambre
535,5,Plataforma
425,5,Radical
427,5,La Era Del Diamante: Manual Ilustrado Para Jov...
448,5,Preguntale Al Economista Camuflado
473,5,Un Paseo Aleatorio Por Wall Street
479,5,Ruido de Pasos


### ZISBN ZASIN


In [100]:
b_h_dataframe.loc[b_h_dataframe["ZISBN"].notnull()][["ZISBN","ZTITLE"]]

,ZISBN,ZTITLE
7,8423316777,Contra El Infinito
28,8496968812,Worl Word Z
37,9685960437,El Sol Desnudo
86,84-45-07068-1,"Sirenas de Titan, Las"
121,8448040317,Diario de Una Invasion Zombie
135,8401496713,Visiones de Robot
174,8420699683,El Nacimiento de Los Estados Unidos
178,8433973282,El Restaurante Del Fin Del Mundo
218,8496829006,El Codigo de Da Vinci
252,8435015955,Blade Runner


In [101]:
b_h_dataframe.loc[b_h_dataframe["ZASIN"].notnull() & 
                      b_h_dataframe["ZISBN"].isnull()][["ZASIN","ZISBN","ZTITLE"]]

,ZASIN,ZISBN,ZTITLE
47,B007OWGBAK,NaN,Tiempo de Marte
341,B004VY3P8K,NaN,"Cluetrain Manifesto, The"
494,B009ZJAMAU,NaN,Angeles Y Demonios
518,B00GZ0RGME,NaN,La Voz de Los Muertos
564,B005EHQ5TO,NaN,Cumbres Borrascosas
803,B003B07B9Y,NaN,El Fin de La Eternidad
850,B0078ZUYH4,NaN,En El Oceano de La Noche
890,B00GI27B74,NaN,El Juego de Ender
1017,B005D5FQMO,NaN,(A DANCE WITH DRAGONS: A SONG OF ICE AND FIRE:...
1020,B003N1D2AS,NaN,"S.D Levitt's,S.J.Dubner'sFreakonomics (Revised..."


### ZGENRE
This value must be clean

In [102]:
print(b_h_dataframe.groupby("ZGENRE")["ZGENRE"].count().sort_values().to_string())

ZGENRE
runners wordl               1
Literatura                  1
novela oeste                1
Politica; Novela            1
Politicia                   1
Guias                       1
George rr martin            1
Fotografia                  1
Relatos infantis            1
Religion                    1
Erotico                     1
Embarazo                    1
Manual                      1
Educational                 1
Tratado Educacion           1
Literatura galega           1
Diseño                      1
Autodesarrollo              1
novela                      1
kayak                       1
dieta                       1
Varios                      1
biologia                    1
Computer                    1
Cociña                      1
Deportes                    1
Desarrollo Personal         1
Desarrollo personal         1
Viñetas                     2
Software                    2
Tecnologia                  2
Quimica                     2
manual kayak                2
nov

In [103]:
b_h_dataframe[b_h_dataframe["ZGENRE"]=="Articulos"]["ZTITLE"]

144    Madrid. El Advenimiento de La Republica
559                                100 Artigos
568                                  Articulos
Name: ZTITLE, dtype: object

### ZLANGUAGES
This values must be clean

In [104]:
print(b_h_dataframe.groupby("ZLANGUAGES")["ZLANGUAGES"].count().to_string())

ZLANGUAGES
CASTELAN                4
Castelan              853
Castelan e frances      1
Castellano              2
Catalán                 1
English                41
English; Spanish        2
Español               118
Frances                 2
French                  1
French; Spanish         1
Galeg                   1
Galego                 83
Galician                3
Gallego                 2
Ingles                 25
Inglés                 14
Spanish                30
español                 1


### ZAUTHOR
NaN values must be clean

In [105]:
b_h_dataframe.loc[b_h_dataframe["ZAUTHOR"].isnull()][["ZAUTHOR","ZTITLE"]]

,ZAUTHOR,ZTITLE
52,NaN,Mitologia
145,NaN,Los Premios Hugo 1955-1961
273,NaN,El Hombre Que Vendio Su Ferrari
327,NaN,Manual Analisis Redes
639,NaN,Manual Practicas Electronia Xeral E Analoxica
849,NaN,El Libro de Los Muertos
1035,NaN,"[In the Plex: How Google Thinks, Works, and Sh..."
1043,NaN,el principio de dilbert
1048,NaN,the walking dead days gone bye
1060,NaN,simiocracia


### ZCOLLECTION

In [106]:
print(b_h_dataframe.groupby("ZCOLLECTION")["ZCOLLECTION"].count().to_string())

ZCOLLECTION
/ Literatura                                 1
1                                            1
277                                          1
380                                          1
Actualidad                                   1
Arbore                                       1
Arealonga                                    2
Barco DeVapor                                1
Basica                                       1
Best-Sellers                                 7
Bibliotaca Galega 120                        1
Biblioteca                                   3
Biblioteca Alberti                           1
Biblioteca Basica Da Cultura Galega          2
Biblioteca Blasco Ibañez                     1
Biblioteca Borges                            1
Biblioteca Breve                             1
Biblioteca Camus                             2
Biblioteca Ciencia Ficccion                  1
Biblioteca Juvenil                           1
Biblioteca Lem                               1
B

### ZFORMAT

In [107]:
print(b_h_dataframe.groupby("ZFORMAT")["ZFORMAT"].count().to_string())

ZFORMAT
Audio CD                     1
Ebook Kindle                10
Hard Cover                   1
Hardcover                   18
Kindle Ebook                 2
Kindle Edition               7
Kindle eBook                 2
Libro de bolsillo            1
Mass Market Paperback        1
Paperback                   76
Perfect Paperback           11
Tapa Blanda (Reforzada)      1
Tapa blanda                106
Tapa blanda (reforzada)      3
Tapa dura                   12
Texto grande                 1
Unknown Binding              4
Versión Kindle              11
eBook Kindle                43
true                        13
Álbum                        1


# PREPROCESS

Clean **ZGENRE, ZLANGUAGES, ZFORMAT**

In [133]:
# List values to replace
# Format : Column mame, value to replace, new value
LST_REPLACE_VALUE = [["ZRATING", "Sokyo Ono", '1'],
               ["ZGENRE", "George rr martin", "Ciencia-Ficcion"],
               ["ZGENRE", "Fantasy", "Fantastica"],
               ["ZGENRE", "Literatura Fantastica", "Fantastica"],
               ["ZGENRE", "No", "Novela"],
               ["ZGENRE", "Religion", "Relixion"],
               ["ZGENRE", "Viajes", "Viaxes"],
               ["ZGENRE", "Guias", "Viaxes"],
               ["ZGENRE", "Viñeta", "Comic"],
               ["ZGENRE", "biologia", "Biologia"],
               ["ZGENRE", "historia", "Historia"],
               ["ZGENRE", "kayak", "Kayak"],
               ["ZGENRE", "manual kayak", "Kayak"],
               ["ZGENRE", "marketing", "Marketing"],
               ["ZGENRE", "novala", "Novela"],
               ["ZGENRE", "novela oeste", "Novela"],
               ["ZGENRE", "Best-Seller", "Novela"],
               ["ZGENRE", "runners wordl", "Deportes"],
               ["ZGENRE", "Divulgacion Cientifica", "Ciencia"],
               ["ZGENRE", "Quimica", "Ciencia"],
               ["ZGENRE", "Relatos infantis", "Relatos"],
               ["ZGENRE", "Manual", "Desarrollo personal"],
               ["ZGENRE", "Educational", "Educacion"],
               ["ZGENRE", "Tratado Educacion", "Educacion"],
               ["ZLANGUAGES", "CASTELAN", "Castelan"],
               ["ZLANGUAGES", "Castellano", "Castelan"],
               ["ZLANGUAGES", "Español", "Castelan"],
               ["ZLANGUAGES", "Spanish", "Castelan"],
               ["ZLANGUAGES", "español", "Castelan"],
               ["ZLANGUAGES", "Frances", "French"],
               ["ZLANGUAGES", "Galeg", "Galego"],
               ["ZLANGUAGES", "Galician", "Galego"],
               ["ZLANGUAGES", "Gallego", "Galego"],
               ["ZLANGUAGES", "Ingles", "English"],
               ["ZLANGUAGES", "Inglés", "English"],
               ["ZFORMAT", "Audio CD", "Tapa dura"],
               ["ZFORMAT", "Ebook Kindle", "Ebook"],
               ["ZFORMAT", "Kindle Ebook", "Ebook"],
               ["ZFORMAT", "Kindle eBook", "Ebook"],
               ["ZFORMAT", "Kindle Edition", "Ebook"],
               ["ZFORMAT", "Versión Kindle", "Ebook"],
               ["ZFORMAT", "eBook Kindle", "Ebook"],
               ["ZFORMAT", "Hardcover", "Hard Cover"],
               ["ZFORMAT", "Tapa dura", "Hard Cover"],
               ["ZFORMAT", "Libro de bolsillo", "Paperback"],
               ["ZFORMAT", "Mass Market Paperback", "Paperback"],
               ["ZFORMAT", "Perfect Paperback", "Paperback"],
               ["ZFORMAT", "Tapa Blanda (Reforzada)", "Paperback"],
               ["ZFORMAT", "Tapa blanda (reforzada)", "Paperback"],
               ["ZFORMAT", "Tapa blanda", "Paperback"],
               ["ZFORMAT", "Texto grande", "Paperback"],
               ["ZFORMAT", "Unknown Binding", "Paperback"],
               ["ZFORMAT", "true", "Paperback"],
              ]

for replace_value in LST_REPLACE_VALUE:
    [column_name, value_to_replace, new_value] = replace_value
    b_h_dataframe.loc[b_h_dataframe[column_name] == value_to_replace, column_name]= new_value

Remove book with wrong **title**

In [134]:
LST_REMOVE_ROWS = [["ZTITLE","New Book"],
                  ["ZTITLE","the element of statistical learning"]]

for remove_rows in LST_REMOVE_ROWS:
    [column_name, value_to_remove] = remove_rows
    b_h_dataframe = b_h_dataframe.drop(
                        b_h_dataframe[b_h_dataframe[column_name] == value_to_remove].index)

Join **ZISBN and ZASIN**

In [135]:
def row_2_isbn(row):
    if row["ZISBN"] is not None:
        return row["ZISBN"]
    if row["ZASIN"] is not None:
        return row["ZASIN"]
    return None

b_h_dataframe['ISBN 10']=b_h_dataframe.apply(lambda row: row_2_isbn(row), axis=1)

Rename columns to names used by LibIb.

In [136]:
b_h_dataframe = b_h_dataframe.rename(columns=\
                                     {"ZAUTHOR" : "Authors",
                                    "ZTITLE" : "Title",
                                    "ZSUMMARY" :"Description",
                                     "isbn" : "ISBN 10",
                                     "ZGENRE" : "Group",
                                     })

Translate **Status and Rating**

In [137]:
NEW_STATUS=["Not Begun", #0 
            "Not Begun", #1 
            "In Progress",  #2
            "Abandoned",  #3
            "Completed"  ] #4

def get_row_new_status(row):
    return (NEW_STATUS[row["ZSTATUS"]])

b_h_dataframe["Status"] = b_h_dataframe.apply(lambda row:get_row_new_status(row), axis=1)

In [138]:
b_h_dataframe["ZRATING"] = b_h_dataframe["ZRATING"].fillna(0.0).astype(int)
b_h_dataframe["Rating"] = b_h_dataframe["ZRATING"]/6*5

**ZLANGUAGES,ZCOLLECTION,ZFORMAT** are imported as TAGS

In [139]:
COLUMNS_2_TAGS = [ "ZLANGUAGES", "ZCOLLECTION","ZFORMAT"]

def add_tag(tags, tag_to_add):
    if len(tags)>0:
        return tags + "," + str(tag_to_add)
    else:
        return tag_to_add
def get_row_tags(row):
    status = ""
    for column in  COLUMNS_2_TAGS:
        if row[column] is not pd.np.nan :
            status = add_tag(status, str(row[column]))
    return status  

b_h_dataframe["Tags"]=b_h_dataframe.apply(lambda row:get_row_tags(row), axis=1)

In [140]:
COLUMNS_TO_DROP = ["ZSTATUS", "ZRATING", "ZLENGTH"] + COLUMNS_2_TAGS 
b_h_dataframe.drop(labels=COLUMNS_TO_DROP, axis=1, inplace=True)

In [141]:
def replace_value(dataframe, title, new_value, column):
    dataframe.loc[dataframe["Title"] == title, 
                  column]= new_value

def replace_lst_values(dataframe, lst_value_to_change):
    for value_to_change in lst_value_to_change:
        title, new_value, column = value_to_change
        replace_value(dataframe, title, new_value, column)

## Iterate to correct Author problems

In [145]:
b_h_dataframe[b_h_dataframe["Authors"].isnull()][["Title","Authors"]]

,Title,Authors
1306,,NaN


In [143]:
# List values to replace
# Format : Title of the book to change, New value, Column to change
LST_CHANGE_AUTHOR_BY_TITLE = [
    ["El Hombre Que Vendio Su Ferrari","EL MONJE QUE VENDIO SU FERRARI", "Title"],
    ["EL MONJE QUE VENDIO SU FERRARI","ROBIN SHARMA", "Authors"],
    ["el principio de dilbert","Scott Adams", "Authors"],
    ["the walking dead days gone bye","ROBERT KIRKMAN", "Authors"],
    ["simiocracia","Aleix Saló", "Authors"],
    ["Los miedos del pequeño","Edurne Romo", "Authors"],
    ["Enseñarle a ser sociable","Edurne Romo", "Authors"],
    ["Applied Evolutionary Psychology","S. Craig Roberts", "Authors"],
    ["Old Mars","George R. R. Martin", "Authors"],
    ["el indice del miedo","Robert Harris", "Authors"],
    ["MUNDO DE HIELO Y FUEGO","George R. R. Martin", "Authors"],
    ["LA guia del corredor","Runner's World", "Authors"],
    ["Sintoismo. La via de los kami","Sokyo Ono", "Authors"],
    ["Los Premios Hugo 1955-1961","Varios", "Authors"],
    ["Mitologia","Philip Wilkinson", "Authors"],
    ["Manual Analisis Redes","Desconocido", "Authors"],
    ["Manual Practicas Electronia Xeral E Analoxica","Desconocido", "Authors"],
    ["El Libro de Los Muertos","WALLIS BUDGE", "Authors"],
    ["[In the Plex: How Google Thinks, Works, and Shapes Our Lives by Levy, Steven]Author [Hardcover]",
         "Levy, Steven", 
         "Authors"],
    ["[In the Plex: How Google Thinks, Works, and Shapes Our Lives by Levy, Steven]Author [Hardcover]",
         "In the Plex: How Google Thinks, Works, and Shapes Our Lives", "Title"],
    ["escuchale y que te escuche","Desconocido", "Authors"],
    ["Machine learning refined","Jeremy Watt", "Authors"],
    ["Machine learning refined",
         "Machine Learning Refined: Foundations, Algorithms, and Applications", 
         "Title"],
    ["midlife crisis","Albert Rogers", "Authors"],
    ["The Elements of Statistical Learning by Hastie, Trevor, Tibshirani, Robert, Friedman, Jerome. (Springer,2009) [Hardcover] 2ND EDITION",
         "Trevor Hastie", 
         "Authors"],
    ["The Elements of Statistical Learning by Hastie, Trevor, Tibshirani, Robert, Friedman, Jerome. (Springer,2009) [Hardcover] 2ND EDITION",
         "The Elements of Statistical Learning", 
         "Title"],
    ["Genome. The autobiography of a spices",
         "Genome: The Autobiography of a Species in 23 Chapters", 
         "Title"],
    ["Genome: The Autobiography of a Species in 23 Chapters",
         "Matt Ridley ", 
         "Authors"],
    ["La fuga de logan","William F. Nolan", "Authors"],
    ["Maiking thigs happen","Making things happen", "Title"],
    ["Making things happen", "Ronald T. Kellog","Authors"],
    ["How to al almost everything and still win big", "Ronald T. Kellog","Authors"],
    ["How to al almost everything and still win big", "Scott Adams","Authors"],
    ["How to al almost everything and still win big",
         "How to Fall almost everything and still win big", 
         "Title"],
    
]

In [144]:
replace_lst_values(b_h_dataframe, LST_CHANGE_AUTHOR_BY_TITLE)

## ISBN

In [146]:
#Only run this if you have a valid google api key.
import GoogleBookApiKey as google_key
isbn.config.add_apikey("goob", google_key.KEY)

In [10]:
def get_isbn_from_isbn_10(row):
    try:
        if row["ISBN 10"].notnull():
            isbn_10 = re.sub("[^0-9]", "", row["ISBN 10"])
            if len(isbn_10) == 13:
                return isbn_10
            return isbn.meta(row["ISBN 10"])["ISBN-13"]
        else:
            return None
    except:
        return None

def get_isbn_from_goom(row):
    # Get isbn from Google Books
    try:
        response = isbn.goom(row["Title"])
        if len(response) == 0:
            return None
        return response[0]["ISBN-13"]
    except:
        return None    
    
    
def get_isbn_from_row(row, debug=False):
    if ("isbn_13") in row:
        if pd.notnull(row["isbn_13"]):
            return row["isbn_13"]
    if debug:
        print("From web {}".format(row["Title"]))
    isbn=get_isbn_from_isbn_10(row)
    if isbn is not None:
        return isbn
    return get_isbn_from_goom(row)

def print_percent(dataframe):
    print("ISBN OK {}. Falta {}".format(
                    len(dataframe[dataframe["isbn_13"].notnull()]),
                    len(dataframe[dataframe["isbn_13"].isnull()])
                    )
     )    
    
  
def update_dataframe_isbn(dataframe, debug=False):
    dataframe["isbn_13"] = dataframe.apply(lambda row:get_isbn(row,debug), axis=1)
    print_percent(b_h_dataframe)

In [153]:
update_dataframe_isbn(b_h_dataframe)


CRITICAL:isbnlib.dev.webservice:ISBNLibHTTPError for https://www.googleapis.com/books/v1/volumes?q=%20&fields=items/volumeInfo(title,authors,publisher,publishedDate,language,industryIdentifiers)&maxResults=10 with code 400 [Bad Request]


ISBN OK 1347. Falta 39


## Iterate to correct authors problems

In [154]:
b_h_dataframe   = b_h_dataframe.dropna(axis='rows', subset=["Title"])

In [156]:
print_percent(b_h_dataframe)

ISBN OK 1347. Falta 15


In [163]:
b_h_dataframe[b_h_dataframe["isbn_13"].isnull()][["Title","Authors"]]

,Title,Authors
645,"Nacionalismo, Unha Dexenaracion Do Marxismo",Janus
1306,,NaN


In [161]:
# List values to replace
# Format : Title of the book to change, New value, Column to change
LST_CHANGE_VALUE_BY_TITLE = [
    ["Ciberiana", "Ciberiada", "Title"],
    ["El Vido Del Estio", "El Vino Del Estio", "Title"], 
    ["Criptomomicon II", "Cryptonomicon II", "Title"],
    ["Criptomonicon II", "Cryptonomicon II", "Title"],
    ["Dialectica Eristrica", "Dialectica Eristica", "Title"],
    ["Abellos de Ouro", "Abellas de Ouro", "Title"],
    ["Diblineses", "Dublineses", "Title"],
    ["Veneños", "Venenos", "Title" ],
    ["Venenos", "9788486040895" , "isbn_13"],
    ["Electric Circuir", "Electric Circuit", "Title"],
    ["Electric Circuit", "9780201401004" , "isbn_13"],
    ["Codigo Genetico", "9788498009675", "isbn_13" ],
    ["Crónica de nos", "Crónica de nós", "Title"],
    ["Bug-Dargal", "Bug-Jargal", "Title"],
    ["Constantinopla",  "9788420635330", "isbn_13"],
    ["Peaxes", "9788497829588", "isbn_13"],
    ["Pies descalzos V1", "Pies descalzos 1","Title"],
    ["Egio, El Grajero de Jan, Hoja de Niggle, El Granjero de Mootonmayor", 
         "Egidio, el granjero de Ham / Hoja de Niggle / El herrero de Wootton Mayor", 
         "Title"],
    [ "Egidio, el granjero de Ham / Hoja de Niggle / El herrero de Wootton Mayor", 
         "9788445072486",
         "isbn_13"],
    ["S.D Levitt's,S.J.Dubner'sFreakonomics (Revised and Expanded))(Freakonomics , A Rogue EconomistExploresHidden Sideof Everything [Hardcover](2006)", 
         "Freakonomics", 
         "Title"],
    ["Freakonomics", "9788466625128", "isbn_13"],
    ["Cita Con Rama", "9788435021043", "isbn_13"],
    ["A esmorga", "9788471544261", "isbn_13"],
    ["People Over Profit by Dale Partridge (14-May-2015) Paperback", 
         "People Over Profit: Break the System, Live with Purpose, Be More Successful",
         "Title"],
    ["People Over Profit: Break the System, Live with Purpose, Be More Successful",
         "9780718021757",
         "isbn_13"],
    ["Bilo Y Nano", "Bilbo Y Nano", "Title"],
]

In [162]:
replace_lst_values(b_h_dataframe, LST_CHANGE_VALUE_BY_TITLE)
update_dataframe_isbn(b_h_dataframe)

/Users/jotavaladouro/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
CRITICAL:isbnlib.dev.webservice:ISBNLibHTTPError for https://www.googleapis.com/books/v1/volumes?q=%20&fields=items/volumeInfo(title,authors,publisher,publishedDate,language,industryIdentifiers)&maxResults=10 with code 400 [Bad Request]


ISBN OK 1360. Falta 2


/Users/jotavaladouro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# EXPORT

In [166]:
# Be sure isbn is an int, not a float.
b_h_dataframe["isbn_13"] = b_h_dataframe["isbn_13"]\
        .astype(float)\
        .fillna(0.0)\
        .astype(int)
# Export the file
# encoding='utf-8' .- Needed to support some charazters
b_h_dataframe[b_h_dataframe["isbn_13"].notnull()].to_csv(FILE_IMPORT, 
                                                columns=COLUMS_EXPORT,
                                                index = False,
                                                encoding='utf-8')

  0%|          | 0/1386 [24:58<?, ?it/s]
